In [ ]:
#10 M events were generated at PX positon including the isotopes 'Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208'
#skipEThreshold: 0.1
#Each isotopes 40 seeds;each simulation: 250K events
# 38 failed for Th228 and 27 failed for Po216
#Total root files in g4 215
#Total root files in recon 211
#comparing with the result in the 5 th slide
#https://docs.google.com/viewer?url=https%3A%2F%2Fnexowiki.llnl.gov%2Fimages%2Fa%2Fa1%2FScott_Schwartz_Th228_Location_Reconstruction_Study.pdf

In [ ]:
#libraries
import uproot
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
import scienceplots
plt.rcParams['figure.figsize'] = [30, 20]
sns.set(rc={'figure.figsize':(30,20)})
#sns.set(font_scale=1.0)
sns.set_theme(style="darkgrid")

# DATA FILES

In [ ]:
#cuts
apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'


dec102020_filter='(standoff>100) & passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < (1.077 * m_nQ + 313)) & (m_nOPCal > (0.597 * m_nQ - 216)) & ~NESTBugFound & ~NearAnodeBugFound & m_DNNvalue>0.85)'
ja_filter='(standoff>100) & m_DNNvalue>0.85 & passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < (1.077 * m_nQ + 313)) & (m_nOPCal > (0.597 * m_nQ - 216))& (abs(energy-2614)<10)'# & ~NESTBugFound & ~NearAnodeBugFound' & (abs(energy-2614)<10)'
s_filter='''
            standoff>20   &\
            m_DNNvalue>0  &\
            energy>0      &\
            energy<3000   &\
            passed_z_thresh &\
            passed_xy_thresh &\
            (n_x_ch_abovenoise>0) &\
            (n_y_ch_abovenoise>0) &\
            (m_nOPCal< (1.064*m_nQ+703))&\
            (m_nOPCal> (0.644*m_nQ-2411))&\
            ~NESTBugFound &\
            ~NearAnodeBugFound       
         '''

In [ ]:
file='/home/thakur/slac_data/10Mth228/10Mth228_Th228_pos648.5_0_-1022.6.root'

In [ ]:
#file dataframe
f=uproot.open(file+':tree')
df=f.arrays(f.keys(),library='pd')

In [ ]:
df

In [ ]:
#columns
df.columns

In [ ]:
#application of s_filter
df_s=df.query(s_filter)

In [ ]:
df_s

In [ ]:
isotopes=['Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208']

In [ ]:
#isotope coutns before the cut
raw_counts=pd.DataFrame(df.isotope.value_counts().reset_index().values,columns=['isonum','raw_counts'])\
.sort_values(by=['isonum'])\
.reset_index(drop=True)

raw_counts["isotopes"]=raw_counts.isonum.apply(lambda x:isotopes[x-1])
raw_counts=raw_counts[['isonum','isotopes','raw_counts']]
#raw_counts.index=raw_counts.isotopes
#print(raw_counts.to_string())
raw_counts

In [ ]:
#isotope couns after eh cut
cut_counts=pd.DataFrame(df_s.isotope.value_counts().reset_index().values,columns=['isonum','cut_counts'])\
.sort_values(by=['isonum'])\
.reset_index(drop=True)
#cut_counts.insert(1,"isotopes",isotopes)
cut_counts["isotopes"]=cut_counts.isonum.apply(lambda x:isotopes[x-1])
cut_counts=cut_counts[['isonum','isotopes','cut_counts']]
print(cut_counts.to_string())

In [ ]:
merged=raw_counts.merge(cut_counts,how='left')
merged=merged[['isotopes','raw_counts','cut_counts']]
merged=merged.set_index('isotopes')
print(merged.to_string())

In [ ]:
s_counts=[0,0,6,12,0,13570,234244]

In [ ]:
br_list=[1,1,1,1,1,1,0.359]

In [ ]:
merged["s_counts"]=s_counts

In [ ]:
#print(merged[['cut_counts','s_counts']].to_string())
merged

In [ ]:
#applicaiton of br 
merged['cut_counts']=merged['cut_counts']*br_list

In [ ]:
merged

In [ ]:
br=0.3594
br=0.36
int(merged.loc['Tl208'][1]*br)

In [ ]:
#plot for combined
%matplotlib inline
skipEThreshold=0.1
# merged=merged[['isotopes','raw_counts','cut_counts']]
# merged=merged.set_index('isotopes')
ax=merged.plot.bar()
for c in ax.containers:
    # set the bar label
    ax.bar_label(c, fmt='%.0f', label_type='edge')
plt.xlabel('''ISOTOPES''')
plt.title(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV \n')
plt.ylabel('COUNTS')
plt.yscale('log')
txt='test'
#plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
#plt.text(2.5, 10e4, 'cut: '+s_filter.strip().replace(" ",""), ha='center',fontsize='xx-small',color='r',fontweight='bold')

plt.show()

In [ ]:
ax=merged.plot.bar(y='raw_counts')
plt.yscale('log')

In [ ]:
#plot for raw data
skipEThreshold=0.1
plt.figure(figsize=(16,10))
raw_counts=raw_counts.set_index('isotopes')
ax=raw_counts.plot.bar()
for c in ax.containers:
    # set the bar label
    ax.bar_label(c, fmt='%.0f', label_type='edge')
plt.xlabel('ISOTOPES')
plt.title(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV')
plt.ylabel('COUNTS')
plt.yscale('log')
plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
plt.show()

In [ ]:
#plot for cut data
skipEThreshold=0.1
plt.figure(figsize=(16,10))
cut_counts=cut_counts.set_index('isotopes')
ax=cut_counts.plot.bar()
for c in ax.containers:
    # set the bar label
    ax.bar_label(c, fmt='%.0f', label_type='edge')
plt.xlabel('ISOTOPES')
plt.title(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV')
plt.ylabel('COUNTS')
plt.yscale('log')
#plt.savefig(f'g4tree_isotopes_bothskipEThreshold: {skipEThreshold}.pdf')
plt.show()

In [ ]:
#energy graph
# df_iso=df_s.query("isotope==7")`
# df_iso.energy.plot()
#df_iso.plot('energy','weight')
# ene=df_iso.energy
# weight=df_iso.weight
# label_value="test"
#out=ax.hist(df_iso.energy, bins=200, histtype=u'step', weights=weight, density=True, label=label_value)
# plt.show()
df_s.columns

In [ ]:
#sns.relplot(data=df_s,x='energy',y='weight',hue='isotope')
px_energy=df_s.energy
plt.hist(px_energy,200,density=False,histtype='step',facecolor='g',alpha=0.75)
plt.yscale('log')
plt.xlabel('Energy [keV]')
plt.ylabel('Filtered count ')
plt.show()

In [ ]:
iso_names=cut_counts.isonum.apply(lambda x:isotopes[x-1])
iso_names

In [ ]:
bins=int(merged.cut_counts.sum())
bins

In [ ]:
iso_name=['Th228','Ra224','Rn220','Po216','pb212','bi212','tl208']

In [ ]:
df_s.head()

In [ ]:
df_s['iso_name']=df_s['isotope'].map({1:'Th228',2:'Ra224',3:'Rn220',4:'Po216',5:'Pb212',6:'Bi212',7:'Tl208'})

In [ ]:
df_s.head()

In [ ]:
#sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step")
#bins=merged.cut_counts.sum().values
sns.set(rc={'figure.figsize':(20,10)})
#sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
g=sns.histplot(data=df_s, x="energy",hue="iso_name",weights="weight",bins=1000,element="step",fill=False)#,palette=['r','b','g','y','k'])
#plt.legend(labels=iso_names)
g.set_xlabel('Energy [keV]')
g.set_ylabel('Count')
plt.yscale("log")


In [ ]:
fig, axes = plt.subplots(1, 2,gridspec_kw={'width_ratios':[3,1]})
sns.set(rc={'figure.figsize':(20,10)})
#sns.histplot(data=df_s, x="energy",hue="isotope",bins=200,element="step",log_scale=True,fill=False)
g=sns.histplot(data=df_s, x="energy",hue="iso_name",weights="weight",bins=1000,element="step",fill=False,ax=axes[0])#,palette=['r','b','g','y','k'])
#plt.legend(labels=iso_names)
g.set_xlabel('Energy [keV]')
g.set_ylabel('Counts')
g.set_yscale("log")

#g1=sns.barplot(data=merged,x='
ax=merged[["raw_counts","cut_counts"]].plot.bar(ax=axes[1])
for c in ax.containers:
    # set the bar label
    ax.bar_label(c, fmt='%.0f', label_type='edge')
ax.set_xlabel('Isotopes')

ax.set_ylabel('Counts')
ax.set_yscale('log')

plt.suptitle(f'Isotope counts in g4tree for 10M Th228 at PX for skipEThreshold: {skipEThreshold} keV \n')
plt.savefig('energyandisotopes.pdf')

plt.show()

In [ ]:
iso=['Th228','Ra224','Rn220','Po216','pb212','bi212','tl208']
col=['m','y','g','k','c','m','b']
filtered_th228=df_s.copy()
print(f"Total rows: {filtered_th228.shape[0]}")
for i in [1,2,3,4,5,6,7]:
    #print(non_filtered_th228_px.head())
    df_new=filtered_th228[filtered_th228.isotope==i]
    if df_new.empty:continue
    #print(df_new.head())
    px_energy=df_new.energy;weight=df_new.weight
    plt.hist(px_energy,2000,density=False,weights=weight,histtype='step',alpha=0.95,label=iso[i-1],color=col[i-1])
    plt.yscale('log')
    plt.xlabel('Energy [keV]')
    plt.ylabel('Raw count ')
    plt.legend()
plt.title("SS Energy for Th228 at all positions")
plt.show()